
[Dataset](https://drive.google.com/file/d/1wWsrN2Ep7x6lWqOXfr4rpKGYrJhWc8z7/view)

# New Section

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
Root = "/content/drive/MyDrive/speech-emotion-recognition-ravdess-data"
os.chdir(Root)

In [ ]:
ls

Actor_01/  Actor_06/  Actor_11/  Actor_16/  Actor_21/
Actor_02/  Actor_07/  Actor_12/  Actor_17/  Actor_22/
Actor_03/  Actor_08/  Actor_13/  Actor_18/  Actor_23/
Actor_04/  Actor_09/  Actor_14/  Actor_19/  Actor_24/
Actor_05/  Actor_10/  Actor_15/  Actor_20/  modelForPrediction1.sav


In [ ]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [ ]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [ ]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [ ]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("/content/drive/MyDrive/speech-emotion-recognition-ravdess-data/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [ ]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [ ]:
x_train

array([[-4.09896820e+02,  1.51403370e+01, -2.24304562e+01, ...,
         4.35975147e-03,  1.97707559e-03,  1.23020017e-03],
       [-7.16214111e+02,  3.91477280e+01, -3.95843005e+00, ...,
         3.84286723e-05,  2.59832232e-05,  3.60392332e-05],
       [-6.87601501e+02,  3.93286972e+01, -4.07874537e+00, ...,
         2.73509031e-05,  1.62149136e-05,  9.44456588e-06],
       ...,
       [-5.60119873e+02,  3.76125031e+01, -8.76632881e+00, ...,
         2.18110785e-04,  1.51651751e-04,  5.76054408e-05],
       [-5.88531982e+02,  6.03003578e+01,  1.75049515e+01, ...,
         1.34000598e-04,  6.59308062e-05,  1.95767807e-05],
       [-6.55006836e+02,  2.95730686e+01, -5.46247542e-01, ...,
         1.98667931e-05,  2.57811589e-05,  1.41365526e-05]])

In [ ]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [ ]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [ ]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [ ]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [ ]:
#Predict for the test set
y_pred=model.predict(x_test)

In [ ]:
y_pred

array(['happy', 'happy', 'calm', 'calm', 'disgust', 'calm', 'calm',
       'happy', 'happy', 'happy', 'calm', 'happy', 'calm', 'calm',
       'disgust', 'fearful', 'calm', 'calm', 'calm', 'calm', 'happy',
       'calm', 'fearful', 'fearful', 'happy', 'happy', 'disgust',
       'fearful', 'calm', 'happy', 'calm', 'fearful', 'happy', 'happy',
       'fearful', 'calm', 'fearful', 'fearful', 'happy', 'calm', 'happy',
       'happy', 'happy', 'calm', 'happy', 'calm', 'happy', 'calm',
       'happy', 'calm', 'calm', 'fearful', 'happy', 'fearful', 'calm',
       'fearful', 'fearful', 'happy', 'fearful', 'calm', 'calm', 'happy',
       'happy', 'calm', 'fearful', 'happy', 'calm', 'disgust', 'disgust',
       'calm', 'fearful', 'fearful', 'disgust', 'disgust', 'happy',
       'happy', 'calm', 'calm', 'calm', 'calm', 'disgust', 'calm', 'calm',
       'calm', 'fearful', 'fearful', 'happy', 'happy', 'disgust',
       'fearful', 'fearful', 'happy', 'calm', 'fearful', 'calm',
       'fearful', 'calm

In [ ]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 67.71%


In [ ]:
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
f1_score(y_test, y_pred,average=None)

array([0.8057554 , 0.46753247, 0.63888889, 0.6875    ])

In [ ]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,happy,happy
1,happy,happy
2,calm,calm
3,calm,calm
4,fearful,disgust
5,calm,calm
6,calm,calm
7,happy,happy
8,disgust,happy
9,happy,happy


In [ ]:
import pickle
# Writing different model files to file
with open( 'modelForPrediction1.sav', 'wb') as f:
    pickle.dump(model,f)

In [ ]:
filename = 'modelForPrediction1.sav'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature("/content/drive/MyDrive/speech-emotion-recognition-ravdess-data/Actor_01/03-01-04-02-01-01-01.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

array(['calm'], dtype='<U7')

In [ ]:
feature

array([[-6.62522827e+02,  5.73402939e+01,  4.32973623e-01,
         2.00030117e+01, -2.07569167e-01, -4.27961230e-01,
        -6.63846588e+00, -1.11645699e+01, -4.32162189e+00,
        -1.56606269e+00, -3.81368446e+00,  3.35571408e-01,
        -6.98173809e+00,  1.60643399e+00, -3.09381437e+00,
        -4.52872086e+00, -3.60219216e+00, -2.88114697e-01,
        -4.93534613e+00, -1.53959382e+00, -2.68463826e+00,
        -5.73684072e+00, -8.06470215e-01, -2.98968887e+00,
        -1.57019103e+00, -2.88928330e-01, -1.47611296e+00,
        -1.73522747e+00, -2.35914016e+00, -2.41390085e+00,
        -3.17594552e+00, -3.44330978e+00, -1.64540672e+00,
        -2.54550982e+00, -3.32737970e+00, -3.15788412e+00,
        -4.27377415e+00, -1.49568295e+00, -2.88858271e+00,
        -2.01887202e+00,  6.93266869e-01,  6.52378678e-01,
         6.13452792e-01,  6.30414903e-01,  6.69092953e-01,
         6.70851529e-01,  6.70543134e-01,  7.10303068e-01,
         7.13138282e-01,  7.42537498e-01,  7.79047966e-0